# Pandas training

Before you work with pandas library, you need to install following Python packages - <strong>pandas</strong>, <strong>openpyxl</strong> (to export data to Excel), <strong>sqlalchemy</strong> and <strong>cx_Oracle</strong> (to create the engine to connect to the database), <strong>bs4</strong>, <strong>lxml</strong>, <strong>html5lib</strong> (loading data from web)

In [1]:
import pandas as pd
from sqlalchemy import create_engine # to connect to the Oracle DB
import getpass # Python's standard library

# Reading files
Additional source: https://pandas.pydata.org/docs/getting_started/intro_tutorials/02_read_write.html.<br/>
Two types of objects:
* DataFrame
* Series

## read_csv function

In [2]:
df_vakances_raw = pd.read_csv(filepath_or_buffer='vakances-2022-05-04.csv')
print(f"{type(df_vakances_raw)=}")

type(df_vakances_raw)=<class 'pandas.core.frame.DataFrame'>


## read_excel function

In [3]:
df_xlsx_vakances = pd.read_excel(io='vakances-2022-05-04.xlsx')
print(f"{df_xlsx_vakances.shape=}")

df_xlsx_vakances.shape=(1478, 14)


## read_sql function
Engines: https://docs.sqlalchemy.org/en/14/core/engines.html

In [7]:
ci = {"user":"user_name", "host":"hostname", "port":1521, "database":"db_name"}
psw = getpass.getpass(f'Enter password to connect to the {ci["database"]} database: ')  
o_engine = create_engine(f'oracle://{ci["user"]}:{psw}@{ci["host"]}:{ci["port"]}/{ci["database"]}') # need also cx_Oracle package

Enter password to connect to the db_name database:  ····


In [6]:
df_db_sample = pd.read_sql(sql="""
    SELECT *
    FROM SCHEMA.TABLE_NAME
    WHERE DATE_COLUMN >= SYSDATE - 7
    ORDER BY DATE_COLUMN, COL1, COL2 
    """,
    con=o_engine)

print(df_db_sample.info()) # Liela saraksta gadījumā nerāda katras kolonnas info.

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1272 entries, 0 to 1271
Columns: 126 entries, d_date to dt_created
dtypes: datetime64[ns](3), float64(28), int64(43), object(52)
memory usage: 1.2+ MB
None


## read_json function

In [8]:
dfjson = pd.read_json("https://jsonplaceholder.typicode.com/todos")
dfjson

,userId,id,title,completed
0,1,1,delectus aut autem,False
1,1,2,quis ut nam facilis et officia qui,False
2,1,3,fugiat veniam minus,False
3,1,4,et porro tempora,True
4,1,5,laboriosam mollitia et enim quasi adipisci qui...,False
...,...,...,...,...
195,10,196,consequuntur aut ut fugit similique,True
196,10,197,dignissimos quo nobis earum saepe,True
197,10,198,quis eius est sint explicabo,True
198,10,199,numquam repellendus a magnam,True


In [15]:
# Population density, area and population of countries and dependencies in Europe
dfhtml = pd.read_html('https://en.wikipedia.org/wiki/Area_and_population_of_European_countries')[1]
dfhtml

,Name,Population density(/km2),Area(km2),Population
0,Monaco,18960.0,2.02,38300
1,Gibraltar (UK),5011.0,6.7,33573
2,Vatican City,1684.0,0.49[a],825
3,Malta,1505.0,316,475701
4,Guernsey (UK),955.0,65,62063
...,...,...,...,...
56,Finland,16.0,338424,5521533
57,Norway,14.0,385203,5323933
58,Kazakhstan (European part),5.5,148000,813621
59,Iceland,3.5,102775,355620


# DataFrame/Series basic information

## head method

In [16]:
df_vakances_raw.head(3)
# in R: head(df_vakances_raw)

,Vakances Nr,Aktualizācijas datums,Iestādes reģistrācijas numurs,Vakances nosaukums,Vakances kategorija,Alga no,Alga līdz,Slodzes tips,Darba laika veids,Darba stundas nedēļā,Pieteikšanās termiņš,Attēls,Vieta,Vakances paplašināts apraksts
0,220419-23,2022-04-29,44103015509,OTRREIZĒJO IZEJVIELU ŠĶIROTĀJS,Pakalpojumi,750.0,1000.0,Viena vesela slodze,NaN,NaN,2022-05-20,NaN,"Daibe, Stalbes pag., Cēsu nov.",https://cvvp.nva.gov.lv/#/pub/vakances/319927475
1,220503-43,2022-05-03,44103015509,GADĪJUMA DARBU STRĀDNIEKS,Pakalpojumi,500.0,500.0,Viena vesela slodze,NaN,NaN,2022-05-20,NaN,"Daibe, Stalbes pag., Cēsu nov.",https://cvvp.nva.gov.lv/#/pub/vakances/321151029
2,220503-61,2022-05-03,44103015509,ATKRITUMU SAVĀCĒJS,Pakalpojumi,700.0,800.0,Viena vesela slodze,NaN,NaN,2022-05-20,NaN,"Valmiera, Valmieras nov.",https://cvvp.nva.gov.lv/#/pub/vakances/321154098


## tail method

In [17]:
df_vakances_raw.tail(3)
# in R: tail(df_vakances_raw)

,Vakances Nr,Aktualizācijas datums,Iestādes reģistrācijas numurs,Vakances nosaukums,Vakances kategorija,Alga no,Alga līdz,Slodzes tips,Darba laika veids,Darba stundas nedēļā,Pieteikšanās termiņš,Attēls,Vieta,Vakances paplašināts apraksts
1475,220503-46,2022-05-03,90000043403,IEKŠĒJAIS AUDITORS,Valsts pārvalde,1045.0,1135.0,Viena vesela slodze,Normālais darba laiks,NaN,2022-05-13,NaN,"Lāčplēša iela 2, Valmiera, Valmieras nov.",https://cvvp.nva.gov.lv/#/pub/vakances/321151300
1476,220405-125,2022-04-29,90009226858,ZOBĀRSTS,Veselības aprūpe / Sociālā aprūpe,1000.0,1000.0,Nepilna slodze,Nepilnais darba laiks,NaN,2022-06-01,NaN,"Brīvības iela 29, Dagda, Krāslavas nov.",https://cvvp.nva.gov.lv/#/pub/vakances/319101963
1477,220228-169,2022-04-29,90009226858,FIZIOTERAPEITS,Veselības aprūpe / Sociālā aprūpe,1032.0,1032.0,Viena vesela slodze,Normālais darba laiks,NaN,2022-06-01,NaN,"Brīvības iela 29, Dagda, Krāslavas nov.",https://cvvp.nva.gov.lv/#/pub/vakances/316797306


## sample method

In [18]:
df_vakances_raw.sample(3)

,Vakances Nr,Aktualizācijas datums,Iestādes reģistrācijas numurs,Vakances nosaukums,Vakances kategorija,Alga no,Alga līdz,Slodzes tips,Darba laika veids,Darba stundas nedēļā,Pieteikšanās termiņš,Attēls,Vieta,Vakances paplašināts apraksts
62,220503-412,2022-05-03,90002065000,ĢEODĒZIJAS INŽENIERIS,Valsts pārvalde,930.0,1210.0,Viena vesela slodze,Normālais darba laiks,40.0,2022-05-20,NaN,"Ojāra Vācieša iela 43, Rīga",https://cvvp.nva.gov.lv/#/pub/vakances/321220897
1092,220429-5,2022-04-29,90000014387,FOTO DIZAINA SPECIĀLISTS,Valsts pārvalde,730.0,800.0,Viena vesela slodze,Normālais darba laiks,NaN,2022-05-15,NaN,"Zavoloko iela 8, Rēzekne",https://cvvp.nva.gov.lv/#/pub/vakances/320848636
847,220322-182,2022-04-27,90009756700,SABIEDRĪBAS VESELĪBAS ANALĪTIĶIS,Valsts pārvalde,835.0,1190.0,Viena vesela slodze,Normālais darba laiks,40.0,2022-05-31,NaN,"Duntes iela 20 k-3, Rīga",https://cvvp.nva.gov.lv/#/pub/vakances/318171733


## dtypes attribute

In [19]:
df_xlsx_vakances.dtypes

Vakances Nr                              object
Aktualizācijas datums            datetime64[ns]
Iestādes reģistrācijas numurs             int64
Vakances nosaukums                       object
Vakances kategorija                      object
Alga no                                 float64
Alga līdz                               float64
Slodzes tips                             object
Darba laika veids                        object
Darba stundas nedēļā                    float64
Pieteikšanās termiņš                     object
Attēls                                  float64
Vieta                                    object
Vakances paplašināts apraksts            object
dtype: object

Types:
* int64 - integer
* float64 - float
* object - string
* datetime64[ns] - datetime
* bool - boolean

## info method

In [20]:
df_xlsx_vakances.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1478 entries, 0 to 1477
Data columns (total 14 columns):
 #   Column                         Non-Null Count  Dtype         
---  ------                         --------------  -----         
 0   Vakances Nr                    1478 non-null   object        
 1   Aktualizācijas datums          1478 non-null   datetime64[ns]
 2   Iestādes reģistrācijas numurs  1478 non-null   int64         
 3   Vakances nosaukums             1478 non-null   object        
 4   Vakances kategorija            1478 non-null   object        
 5   Alga no                        1478 non-null   float64       
 6   Alga līdz                      1478 non-null   float64       
 7   Slodzes tips                   1178 non-null   object        
 8   Darba laika veids              798 non-null    object        
 9   Darba stundas nedēļā           390 non-null    float64       
 10  Pieteikšanās termiņš           1478 non-null   object        
 11  Attēls           

## shape attribute

In [21]:
print(df_vakances_raw.shape) # (No. of rows, No. of columns) as tuple 
print(type(df_vakances_raw.shape))
# in R: dim(df_vakances_raw)

(1478, 14)
<class 'tuple'>


## describe method

In [22]:
print(df_vakances_raw.describe())
# count, mean, std, min, 25%, 50%, 75%, max

       Iestādes reģistrācijas numurs       Alga no     Alga līdz  \
count                   1.478000e+03   1478.000000   1478.000000   
mean                    7.042215e+10    916.101633   1046.719901   
std                     2.502729e+10    548.359565    656.970523   
min                     1.119010e+08      2.720000      2.840000   
25%                     4.090004e+10    619.610000    700.000000   
50%                     9.000003e+10    887.500000   1000.000000   
75%                     9.000081e+10   1093.000000   1286.000000   
max                     9.001163e+10  10280.000000  12700.000000   

       Darba stundas nedēļā  Attēls  
count            390.000000     0.0  
mean              35.115385     NaN  
std               10.731810     NaN  
min                0.000000     NaN  
25%               40.000000     NaN  
50%               40.000000     NaN  
75%               40.000000     NaN  
max               48.000000     NaN  


In [23]:
print(df_vakances_raw.describe(include=['O'])) # statistika non-numeric kolonnām
# count, unique, top, freq

       Vakances Nr Aktualizācijas datums     Vakances nosaukums  \
count         1478                  1478                   1478   
unique        1478                    49                    398   
top      220419-23            2022-05-02  MĀSA (MEDICĪNAS MĀSA)   
freq             1                   181                     42   

       Vakances kategorija         Slodzes tips      Darba laika veids  \
count                 1478                 1178                    798   
unique                  20                    2                      4   
top        Valsts pārvalde  Viena vesela slodze  Normālais darba laiks   
freq                   393                 1009                    566   

       Pieteikšanās termiņš                          Vieta  \
count                  1478                           1478   
unique                   50                            669   
top              2022-05-31  Vasarnīcu iela 20, Daugavpils   
freq                    198                  

## columns property

In [24]:
print(f'{type(df_vakances_raw.columns)}') # 'pandas.core.indexes.base.Index'
print(df_vakances_raw.columns)

print(f'\n{type(df_vakances_raw.columns.values)}') # 'numpy.ndarray'
print(df_vakances_raw.columns.values)

print(f'\n{type(df_vakances_raw.columns.values.tolist())}') # 'list'
print(df_vakances_raw.columns.values.tolist())
# Same as: df_vakances_raw.columns.tolist()

<class 'pandas.core.indexes.base.Index'>
Index(['Vakances Nr', 'Aktualizācijas datums', 'Iestādes reģistrācijas numurs',
       'Vakances nosaukums', 'Vakances kategorija', 'Alga no', 'Alga līdz',
       'Slodzes tips', 'Darba laika veids', 'Darba stundas nedēļā',
       'Pieteikšanās termiņš', 'Attēls', 'Vieta',
       'Vakances paplašināts apraksts'],
      dtype='object')

<class 'numpy.ndarray'>
['Vakances Nr' 'Aktualizācijas datums' 'Iestādes reģistrācijas numurs'
 'Vakances nosaukums' 'Vakances kategorija' 'Alga no' 'Alga līdz'
 'Slodzes tips' 'Darba laika veids' 'Darba stundas nedēļā'
 'Pieteikšanās termiņš' 'Attēls' 'Vieta' 'Vakances paplašināts apraksts']

<class 'list'>
['Vakances Nr', 'Aktualizācijas datums', 'Iestādes reģistrācijas numurs', 'Vakances nosaukums', 'Vakances kategorija', 'Alga no', 'Alga līdz', 'Slodzes tips', 'Darba laika veids', 'Darba stundas nedēļā', 'Pieteikšanās termiņš', 'Attēls', 'Vieta', 'Vakances paplašināts apraksts']


# Filtering and selecting data
## Select single column

In [25]:
s_place = df_vakances_raw['Vieta']
print(f"{type(s_place)=}\n") # 'pandas.core.series.Series'
print(f'{s_place.tail()=}\n') # Last 5 records from s_place Series
print(f'{s_place.shape=}\n') # shape for Series will return tuple with one value - Number of elements (i.e. rows)

type(s_place)=<class 'pandas.core.series.Series'>

s_place.tail()=1473    Parka iela 2, Zilaiskalns, Zilākalna pag., Val...
1474              Rīgas iela 10, Valmiera, Valmieras nov.
1475            Lāčplēša iela 2, Valmiera, Valmieras nov.
1476              Brīvības iela 29, Dagda, Krāslavas nov.
1477              Brīvības iela 29, Dagda, Krāslavas nov.
Name: Vieta, dtype: object

s_place.shape=(1478,)



## Select multiple columns

In [26]:
# ! Multiple column names must be in the list
df_job_place = df_vakances_raw[['Vakances Nr','Vieta']]
print(df_job_place.sample(3))
print(f"\n{type(df_job_place)=}\n") # 'pandas.core.frame.DataFrame'
print(f'{df_job_place.shape=}\n') # (No. of rows, No. of columns) as tuple

     Vakances Nr                                       Vieta
1192  220419-208  Jumaras iela 195, Valmiera, Valmieras nov.
311    220131-15                       Kleistu iela 28, Rīga
1389  220422-191    Rīgas iela 10, Salaspils, Salaspils nov.

type(df_job_place)=<class 'pandas.core.frame.DataFrame'>

df_job_place.shape=(1478, 2)



## Select all columns but one
### drop method

In [27]:
# .drop() by default does not operate inplace
# do inplace=True if you want to permanently remove column(s)
df_all_cols_but_one = df_xlsx_vakances.drop('Vakances Nr',axis=1)
print(f'{df_all_cols_but_one.shape=}')
#df_vakances_raw.drop(axis=1) # axis=0 (default) - drop by index, axis=1 - drop by column name

df_all_cols_but_one.shape=(1478, 13)


### loc property

In [28]:
# colon (:) is mandatory
# This method don't work if columns are MultiIndex
df_all_cols_but_one = df_xlsx_vakances.loc[:, df_xlsx_vakances.columns != 'Vakances Nr']
print(f'{df_all_cols_but_one.shape=}')

df_all_cols_but_one.shape=(1478, 13)


## Filtering DataFrame (single condition)

In [29]:
df_salary_GT_2000 = df_vakances_raw[df_vakances_raw['Alga no']>2000]
print(df_salary_GT_2000[['Vakances nosaukums','Alga no','Alga līdz']].sort_values('Alga no',ascending=False).head(10))
print(f'{df_salary_GT_2000.shape=}')

                                     Vakances nosaukums  Alga no  Alga līdz
539                                     VALDES LOCEKLIS  10280.0    12700.0
309   INFORMĀCIJAS TEHNOLOĢIJU STRUKTŪRVIENĪBAS VADĪ...   4800.0     5200.0
1027                       ANESTEZIOLOGS, REANIMATOLOGS   3600.0     3600.0
668                                     VALDES LOCEKLIS   3500.0     3500.0
1363                                        OFTALMOLOGS   3500.0     3500.0
1187                       ANESTEZIOLOGS, REANIMATOLOGS   3500.0     3500.0
1026                             TRAUMATOLOGS, ORTOPĒDS   3500.0     3500.0
1323                                    VALDES LOCEKLIS   3486.0     3486.0
1322                                    VALDES LOCEKLIS   3486.0     3486.0
549                                       PROGRAMMĒTĀJS   3200.0     3400.0
df_salary_GT_2000.shape=(32, 14)


## notna method

In [30]:
df_is_wh_types = df_vakances_raw[df_vakances_raw['Darba laika veids'].notna()]
print(df_is_wh_types[['Iestādes reģistrācijas numurs','Darba laika veids']].sort_values('Iestādes reģistrācijas numurs').head(10))
print(f'{df_is_wh_types.shape=}')

     Iestādes reģistrācijas numurs      Darba laika veids
715                      111900989  Normālais darba laiks
55                      2013901280  Normālais darba laiks
51                      2013901280  Normālais darba laiks
52                      2013901280  Normālais darba laiks
53                      2013901280  Normālais darba laiks
54                      2013901280  Normālais darba laiks
49                      2013901280  Normālais darba laiks
56                      2013901280  Normālais darba laiks
57                      2013901280  Normālais darba laiks
50                      2013901280  Normālais darba laiks
df_is_wh_types.shape=(798, 14)


## loc property

In [31]:
df_vakances_raw.loc[1000]
# select row (using loc property) by index.
# 1000 - is a name of the index and not the position

Vakances Nr                                                            220503-345
Aktualizācijas datums                                                  2022-05-03
Iestādes reģistrācijas numurs                                         90000012723
Vakances nosaukums                                               SKOLOTĀJA PALĪGS
Vakances kategorija                                           Izglītība / Zinātne
Alga no                                                                     700.0
Alga līdz                                                                   700.0
Slodzes tips                                                                  NaN
Darba laika veids                                                             NaN
Darba stundas nedēļā                                                          NaN
Pieteikšanās termiņš                                                   2022-05-31
Attēls                                                                        NaN
Vieta           

## isin method

In [32]:
df_is_wh_types = df_vakances_raw[df_vakances_raw['Alga no'].isin([1000,2000])]
# only list-like objects are allowed to be passed to isin()

print(f'{df_is_wh_types.shape=}')

df_is_wh_types.shape=(61, 14)


## Filtering DataFrame (multiple conditions)
When combining multiple conditional statements,
each condition must be surrounded by parentheses ().
Moreover, you can not use or/and but need to use the or operator | and the and operator &.

In [33]:
df_salary_GT_1500 = df_vakances_raw[(df_vakances_raw['Alga no']>1500) &
(df_vakances_raw['Alga no']!=df_vakances_raw['Alga līdz'])]
print(df_salary_GT_1500[['Vakances nosaukums','Alga no','Alga līdz']].sort_values('Alga no').head(3))

                                  Vakances nosaukums  Alga no  Alga līdz
333                                 ĀRSTS (EKSPERTS)   1530.0     1700.0
1044      PAŠVALDĪBAS IESTĀDES VADĪTĀJS /PRIEKŠNIEKS   1534.0     1917.0
943   Neatliekamās medicīnas ĀRSTA PALĪGS/ FELDŠERIS   1550.0     1783.0


In [34]:
# You can save filter result separately and use it later
filter_cond = (df_vakances_raw['Alga no']>1500) & (df_vakances_raw['Alga no']!=df_vakances_raw['Alga līdz'])
print(type(filter_cond))

df_vakances_raw.loc[filter_cond,['Vakances nosaukums','Alga no','Alga līdz']].sort_values('Alga no').head(3)

<class 'pandas.core.series.Series'>


,Vakances nosaukums,Alga no,Alga līdz
333,ĀRSTS (EKSPERTS),1530.0,1700.0
1044,PAŠVALDĪBAS IESTĀDES VADĪTĀJS /PRIEKŠNIEKS,1534.0,1917.0
943,Neatliekamās medicīnas ĀRSTA PALĪGS/ FELDŠERIS,1550.0,1783.0


## Selecting AND filtering DataFrame
The loc/iloc operators are required in front of the selection brackets
When using loc/iloc, the part BEFORE the comma is the rows you want, and the part AFTER the comma is the columns you want to select.<br/><br/>
When using the column names, row labels or a condition expression, use the loc operator in front of the selection brackets [].
For both the part before and after the comma, you can use:
- a single label,
- a list of labels,
- a slice of labels,
- a conditional expression or a colon.
Using a colon specifies you want to select all rows or columns.

In [35]:
salary_filter = df_vakances_raw['Alga no']>1500
column_filter = ['Alga no','Vakances paplašināts apraksts']
df_salary_GT_1500_links = df_vakances_raw.loc[salary_filter,column_filter]
df_salary_GT_1500_links.sort_values('Alga no').head(5)

,Alga no,Vakances paplašināts apraksts
85,1524.0,https://cvvp.nva.gov.lv/#/pub/vakances/320704917
333,1530.0,https://cvvp.nva.gov.lv/#/pub/vakances/319674246
1044,1534.0,https://cvvp.nva.gov.lv/#/pub/vakances/320872167
1414,1550.0,https://cvvp.nva.gov.lv/#/pub/vakances/319216202
291,1550.0,https://cvvp.nva.gov.lv/#/pub/vakances/320775594


# Grouping data
## groupby property

In [36]:
# Save result as a Series (grouping by one column)
grouped_res = df_xlsx_vakances.groupby(["Alga no"])["Alga līdz"].count() # Nerādīs kolonnu "Alga līdz"
print(grouped_res.sort_values(ascending=False).head(5)) # nevajag norādīt kolonnu pie sort_values (jo tādas nav)
print(type(grouped_res))

Alga no
835.0     80
500.0     66
940.0     40
1000.0    39
1500.0    37
Name: Alga līdz, dtype: int64
<class 'pandas.core.series.Series'>


In [37]:
# Save result as a DataFrame (grouping by one column)
# "Alga līdz" ir dubultajās kvadrātiekavās
grouped_res2 = df_xlsx_vakances.groupby(["Alga no"])[["Alga līdz"]].count() # single column dataframe
print(grouped_res2.sort_values("Alga līdz", ascending=False).head(5)) # jānorāda kolonna pie sort_values, savādāk nestrādās
print(type(grouped_res2))

         Alga līdz
Alga no           
835.0           80
500.0           66
940.0           40
1000.0          39
1500.0          37
<class 'pandas.core.frame.DataFrame'>


# Save DataFrame
## Save DataFrame as Excel
https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.to_excel.html <br/>
!!! need to install <strong>openpyxl</strong> package or <strong>xlsxwriter</strong> package (depending which engine is used).
### to_excel method

In [38]:
df_vakances_raw.to_excel(excel_writer='vakances-2022-05-04_modif.xlsx',
sheet_name='vakances',
index=False, # False - do not show index ID's in the first column. Index ID = row ID (starting from 0)
engine = 'openpyxl') # Default. Support also: xlsxwriter

## Save multiple DataFrames in Excel

In [39]:
# If executed again - will overwrite result
with pd.ExcelWriter(path='vakances-2022-05-04_multiple_sheets.xlsx') as df:
        df_vakances_raw.to_excel(excel_writer=df, sheet_name='first_sheet', index=False)
        df_vakances_raw.to_excel(excel_writer=df, sheet_name='second_sheet', index=False)